### Making Data Ready

In [102]:
import folium # type: ignore
import geopandas as gpd # type: ignore
from shapely.geometry import Point # type: ignore
import random 

random.seed(100)


In [103]:
map = "../data/maps/ontario.geojson"

gdf = gpd.read_file(map).to_crs(epsg=3857)
gdf['centroid'] = gdf.geometry.centroid
dummy_map = gdf.explode(ignore_index=True).iloc[[2]]

In [104]:
dummy_map['Boundary'] = 'Prince-Edward-County'
m = folium.Map(location=[44, -77.2], zoom_start=11)
m = dummy_map.explore(m=m, fillopacity=0, fill = False, boundary = True, color = 'black', tooltip = "Boundary")
folium.LayerControl().add_to(m)
# m

In [105]:
polygon = dummy_map.geometry.iloc[0]

def sample_points_in_polygon(polygon, num_points=100):
    points = []
    minx, miny, maxx, maxy = polygon.bounds
    while len(points) < num_points:
        random_point = Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if polygon.contains(random_point):
            points.append(random_point)
    return points

sampled_points = sample_points_in_polygon(polygon, num_points=5)
points_gdf = gpd.GeoDataFrame(geometry=sampled_points, crs=dummy_map.crs)

In [106]:
if points_gdf.crs.to_string() != "EPSG:4326":
    points_gdf = points_gdf.to_crs(epsg=4326)
if dummy_map.crs.to_string() != "EPSG:4326":
    dummy_map = dummy_map.to_crs(epsg=4326)

In [107]:
for idx, row in points_gdf.iterrows():
    point = row.geometry
    folium.CircleMarker(
        location=[point.y, point.x],  # (lat, lon)
        radius=3,
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.7
    ).add_to(m)

folium.LayerControl().add_to(m)
# m

In [108]:
import pandas as pd
import geopandas as gpd
import folium
from shapely.wkt import loads

# Load highway data
file_path_highway = "../data/raw/shapefile_data.csv"
df_highway = pd.read_csv(file_path_highway)
df_highway["geometry"] = df_highway["geometry"].apply(loads)
gdf_full = gpd.GeoDataFrame(df_highway, geometry="geometry")

# Filter for Prince Edward County
gdf = gdf_full[(gdf_full["CSDNAME_L"] == "Prince Edward County") | (gdf_full["CSDNAME_R"] == "Prince Edward County")]
gdf = gdf.set_crs(epsg=3347, allow_override=True).to_crs(epsg=4326)

# Load gas station data
file_path_gas = "../data/raw/ontario_gas_stations.csv"
df_gas_station = pd.read_csv(file_path_gas)

# Define bounding box for Prince Edward County
prince_edward_bounds = {
    "min_lat": 43.85,
    "max_lat": 44.10,
    "min_lon": -77.65,
    "max_lon": -76.90
}

# Filter gas stations within the bounding box
df_prince_edward = df_gas_station[
    (df_gas_station["Latitude"] >= prince_edward_bounds["min_lat"]) &
    (df_gas_station["Latitude"] <= prince_edward_bounds["max_lat"]) &
    (df_gas_station["Longitude"] >= prince_edward_bounds["min_lon"]) &
    (df_gas_station["Longitude"] <= prince_edward_bounds["max_lon"])
]

# Create a single combined map
m_combined = folium.Map(location=[44.00, -77.25], zoom_start=10)

# Add highways as GeoJSON
folium.GeoJson(gdf.to_json(), name="Highways").add_to(m_combined)

# Add gas stations as markers
for _, row in df_prince_edward.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=f"{row['Name']} ({row['Brand']})",
        icon=folium.Icon(color="blue", icon="gas-pump", prefix="fa")
    ).add_to(m_combined)

# Display the map
# m_combined


In [109]:
df_prince_edward

,Name,Brand,Latitude,Longitude
1260,Happy Acre,Unknown,44.075716,-77.007036
1559,Goodyear,Unknown,44.096348,-77.589572
1560,Esso,Esso,44.093908,-77.602122
1677,Shell,Shell,44.048199,-77.579688
1907,Free Flow,Unknown,44.061136,-77.600057
2250,Esso,Esso,44.006156,-77.143524
2400,Ultramar,Ultramar,44.021232,-77.135880
2408,Mobil,Mobil,43.999844,-77.160164
2423,Ultramar,Ultramar,44.004752,-77.146795
2486,Circle K,Circle K,44.009343,-77.139184


In [110]:
df_prince_edward = pd.DataFrame(df_prince_edward)

# Create a geometry column from the Longitude and Latitude columns
df_prince_edward['geometry'] = df_prince_edward.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)

# Convert the DataFrame into a GeoDataFrame
df_prince_edward = gpd.GeoDataFrame(df_prince_edward, geometry='geometry', crs="EPSG:4326")
geometry_gdf = gpd.GeoDataFrame(df_prince_edward[['geometry']], geometry='geometry', crs="EPSG:4326")

geometry_gdf.head()

,geometry
1260,POINT (-77.00704 44.07572)
1559,POINT (-77.58957 44.09635)
1560,POINT (-77.60212 44.09391)
1677,POINT (-77.57969 44.0482)
1907,POINT (-77.60006 44.06114)


In [111]:
polygon = dummy_map.geometry.iloc[0]
within_polygon = geometry_gdf.geometry.within(polygon)
geometry_gdf_within = geometry_gdf[within_polygon]

In [112]:
if geometry_gdf_within.crs.to_string() != "EPSG:4326":
    points_gdf.geometry = geometry_gdf_within.to_crs(epsg=4326)
if dummy_map.crs.to_string() != "EPSG:4326":
    dummy_map = dummy_map.to_crs(epsg=4326)

for idx, row in geometry_gdf_within.iterrows():
    point = row.geometry
    folium.Marker(
        location=[point.y, point.x],  # (lat, lon)
        # radius=3,
        # color='Blue',
        # fill=True,
        # fill_color='blue',
        # fill_opacity=0.7,
        popup="Fuel",
        icon=folium.Icon(icon = 'road', color="blue", prefix="fa")
    ).add_to(m)

# folium.LayerControl().add_to(m)
# m

In [113]:
geometry_highway = gpd.GeoDataFrame(gdf[['geometry']], geometry='geometry', crs="EPSG:4326")
# geometry_highway

for idx, row in geometry_highway.iterrows():
    folium.PolyLine(
        locations=[(lat, lon) for lon, lat in row['geometry'].coords],
        color="blue",  
        weight=2.5,    
        opacity=1.0).add_to(m)

In [114]:
m